# [Mars Lander](https://www.codingame.com/ide/puzzle/mars-lander)

My Ynote: "Coding Games - Mars Lander"<br>
[Study "Genetic Algorithm Implementation in Python" with helps of peforth](https://github.com/hcchengithub/GeneticAlgorithmPython/blob/master/Genetic%20Algorithm%20Implementation%20in%20Python.ipynb)

I didn't understand this game at first and that's a big concern to invest my time into developing the program. Thanks to [McKelian](https://www.codingame.com/forum/t/mars-the-game/756) and his [video game](http://www.mckelian.com/index.php/mars/choix) that mimics the Mars Lander puzzle.

<img src=marslander.png>
<img src=ControlPanel.png>

### Goal 
- land on flat ground
- land in a vertical position (tilt angle = 0°)
- vertical speed must be limited ( ≤ 40m/s in absolute value)
- horizontal speed must be limited ( ≤ 20m/s in absolute value)


# 調製 Simulator 成功了！

這套公式成功了。

Initialize

    thrust = 4    # the thrust command 
    rotate = 0    # the rotate command 
    a0 = 0        # absolute angle of the ship 
    pwr0 = 0      # 引擎出力
    gravity = 3.711  # constant on Mars
    p0 = v(2500, 2700)  # Initial position
    s0 = v(0,0)     # Initial speed vector
    g = angle2uv(-180) * gravity  # 重力向量
    step = 0
    
Run

    # f = m.a  因為 m 不變故當成 1 所以 f = a 也就是 force = g 
    # Duration t = 1 一步的時間都是 1 所以移動距離 s0 + (1/2)(force)(t^2) 變成 s0 + (1/2)*(force)
    
    step += 1

    a1 = angle15(a0,rotate)  # 這一步的角度 
    pwr1 = thrust_1(pwr0,thrust)  # 這一步的 引擎推力
    force = g + angle2uv(a1) * pwr1   # force on the ship during this step 
    s1 = s0 + force  # 這一步將達到的速度 s0 + (delta time)(acceleration) 
                     # where delta time is 1, a is force as mentioned above.
    p1 = p0 + s0 + (1/2)*(force)  # where "s0 + (1/2)*(force)" is position delta 這一步將到達的位置

    %f step -->
    %f a0 pwr0 -2 slice -->  # 上一步的 角度 與 引擎出力 0 1 2 3 4
    %f a1 pwr1 -2 slice -->  # 作用在這一步上的 角度 與 引擎出力
    %f force dup -->
    %f ===>      # 重力 與 引擎出力 的合力向量
    %f s1 ===>   # 這一步最後的速度
    %f p1 ===>   # 這一步最後的位置

    a0 = a1
    pwr0 = pwr1
    s0 = s1
    p0 = p1

# 寫出評分 function 

- 到達地表時的 位置、速度、角度、油料 決定得分。
- 與平地的距離要越近越好。

`[X]` 點到線的距離 `distance = np.cross(p2-p1,p3-p1)/np.linalg.norm(p2-p1)` where p2-p1 is the line and p3 is the point. 

`[ ]` 判斷著地的方法就很簡單了，用 ship 的位置來算與每個 surface 的距離，如果到了平地就是降落了，如果到了其他 surface 就是撞毀了。

`[ ]` 降落平地還要看角度(0)、速度(20,40)，不對還是失敗。

`[ ]` 降落平地之後，油料用得少決定勝負。

1. Crashed 或 landed 得分都是與 flate ground 的直線距離 (越近越好) 
   與 B0 = np.linalg.norm(v(7000,3000)) 的差距，最高分就是這個值。

2. landed 還有加分 bonus, 角度越接近 0 越好，Bonus 要比前一個分數高，使得 crash 完全沒得比。
   角度是 -90~90 之間，故 +-90度得 B0 分，而 0度得 2B0 分，推導出: <br>
    `bonus_angle(an) = 2B0 - (B0/90)*abs(an)`

3. 水平速度要 20 以下，垂直速度要 40 以下。同上，水平速度 500 時得 B0 分，而 20 以下時得 2B0 分<br>
    `bonus_sh(sh) = 2B0 - (B0/500)*abs(0 if sh<20 else sh)`<br>
    `bonus_sv(sv) = 2B0 - (B0/500)*abs(0 if sv<40 else sv)`

4. 如果三個都滿足，再加上 fuel 看剩多少。都用完了得 B0 分，都沒用得 2B0 分。

    `bonus_fuel(fuel) = B0 + (B0/Fuel)*(fuel)`


# Laboratory

In [1]:
import numpy as np
from typing import List, Optional, Callable, Tuple, Dict, Union  # OneNote2021 #_python_ #_typing_
GENE = List[int]  # (rotate (-15 ~ +15),power (-1, 0, +1)) 避免無用的值域，下給 Web site 的 command 要轉換成 (rotate(-90 ~ +90), power(0,1,2,3,4))
GENOME = List[GENE]  # Genome 是一個 list 裡面都是 int
STATE = Dict[str,Union[int,float,np.array]]  # x,y,hspeed,power,.. etc 

# Get modules and peforth tools
import random, json, math, copy, IPython
from itertools import permutations  # permutations 數學「排列組合」中的「排列」

%run forth.py

reDef unknown
reDef -->
p e f o r t h    v1.24
source code http://github.com/hcchengithub/peforth
Type 'peforth.ok()' to enter forth interpreter, 'exit' to come back.



In [4]:

# Mars Lander Episode-2

import numpy as np
import math, random


# Utilities

def v(x,y):
    # vectorize to numpy array which is also a vector 
    return np.array([x,y])

def uv(v):
    # Normalize the given vector to an unit vector
    # https://stackoverflow.com/questions/21030391/how-to-normalize-an-array-in-numpy
    norm = np.linalg.norm(v)
    return v if norm==0 else v/norm;

def norm(v):
    # length of a vector, where v (vector) is a np.array
    return np.linalg.norm(v)

def angle2uv(an):
    # Convert the given absolute angle in degrees to a unit vector that represents the angle
    an1 = np.deg2rad(an + 90)  # ? 90 ? Mars Lander ???????????????
                               # ?? g ? Mars Lander ???????? -180 
    return uv(v(math.cos(an1), math.sin(an1))) 

def angle15(a0,rotate):
    # rotate command ??? absolute target angle ????? +-15??
    if rotate > a0:
        result = a0 + min(rotate-a0,15)
    else :
        result = a0 - min(a0-rotate,15)
    return int(np.round(max(min(90,result),-90),0))

def thrust_1(power,thrust):
    # thrust_1 command ?? power ????? +1 or -1 
    result = power
    if thrust > power:
        result += 1
    if thrust < power:
        result -= 1
    return int(np.round(max(min(4,result),0),0))

# Calculate the euclidian distance between an array of points to a line segment in Python without for loop
# https://stackoverflow.com/questions/54442057/calculate-the-euclidian-distance-between-an-array-of-points-to-a-line-segment-in

def lineseg_dists(p, a, b):
    # where p is an array of points, a,b is the line segment
    if np.all(a == b):
        return np.linalg.norm(p - a, axis=1)

    # normalized tangent vector
    d = np.divide(b - a, np.linalg.norm(b - a))

    # signed parallel distance components
    s = np.dot(a - p, d)
    t = np.dot(p - b, d)

    # clamped parallel distance
    h = np.maximum.reduce([s, t, np.zeros(len(p))])

    # perpendicular distance component, as before
    # note that for the 3D case these will be vectors
    c = np.cross(p - a, d)

    # use hypot for Pythagoras to improve accuracy
    return np.hypot(h, c)


# Constants 

land = [v(0,100), v(1000,500), v(1500,1500), v(3000,1000), 
        v(4000+500,150), v(5500-500,150),  # flate ground 要內縮一點，避免 ship 來到與相鄰 surface 的平分線上撞毀被誤判成降落到位。
        v(6999,800)]

flate = [i for i in range(len(land)-1) if land[i][1] == land[i+1][1]][0]  # flate ground left point index of land array 
fuel0 = 5501  # full tank 
p0 = v(2500, 2700)  # Initial position
a0 = 0        # absolute angle of the ship 
pwr0 = 0      # 引擎出力
s0 = v(0,0)   # Initial speed vector
B0 = np.linalg.norm(v(7000,3000))  # bonus base 
gravity = 3.711  # constant on Mars
g = angle2uv(-180) * gravity  # 重力向量

    
# Genetic Algorithm

def crossover(spouse):
    # spouse 有兩條 chromosome, 遍歷其中短的一條，輪流撿 gene 來組成子代，不夠的用長的續接。
    # 傳回值是子代的 chromosome
    if len(spouse[0][1]) >= len(spouse[1][1]): longer, shorter = spouse[0][1], spouse[1][1]
    else: longer, shorter = spouse[1][1], spouse[0][1]
    shorter_len = len(shorter)
    cross = []
    for i in range(shorter_len): 
        cross.append(spouse[i%2][1][i])
    return cross + longer[i+1:]

# 光用 crossover 才兩代就到了「高原」不會再進步了，這麼快！所以要加上 mutation 才行。參考 Study "Genetic Algorithm Implementation in Python" with helps of peforth 學到了這個方法
# np.random.uniform(-15, 15, 1)  
# np.random.uniform(-1, 1, 1)  
# 本來的加上這個亂數就是 mutation 了！我想只給子代的加上 mutation 以防破壞到好的父代。

# 基因突變 
def mutation(gene):
    # mutation happens on a gene
    delta_rotate = np.random.uniform(-15, 15, 1)  
    delta_power  = np.random.uniform(-1, 1, 1)  
    return (gene[0] + delta_rotate, gene[1] + delta_power)


# The Mars shuttle lander
# 做出一個 class 叫作 Ship, 有油料、位置、速度、角度、引擎出力、軌跡、
# 從開始到落地的一長串 chromosome 也就是一長串的基因 genes (rotate,power)、
# 還有 score。

class Ship:
    def __init__(self, p0, s0, a0, pwr0, fuel0):
        self.inTheAir = True
        self.fuel0 = self.fuel = fuel0
        self.p0 = p0
        self.s0 = s0
        self.a0 = a0
        self.pwr0 = pwr0
        self.trace = []
        self.chromosome = []
        self.score = 0
        
    def make_score(self):
        distances = [lineseg_dists([self.p0], land[i], land[i+1])[0] for i in range(len(land)-1)]  # 與所有 surface 的距離
        peforth.bp(1,locals())
        shortest = np.argmin(distances)  # 最小的 index 
        self.inTheAir = (0 <= np.cross(land[shortest+1]-land[shortest],np.round(self.p0,0)-land[shortest+1]))
                        # 這裡要 大於/等於 0 而不是只 大於 0 ，這純粹是看 CodinGame 怎麼寫法。
        self.score = B0 - distances[flate]  # distance to target 
        # assert self.p0[1] > 200, "close to the problem point"
        if not self.inTheAir:
            if shortest != flate : self.score = -1  # 撞毀了
            else:
                # bonus1 已經降落目的地，通通有獎
                self.score += B0 + 40000  # 速度各減 20000 假設最高速 200 的平方  
                # bonus2 angle the smaller the better 
                self.score -= (B0/90)*abs(self.a0)
                # bonus3 horizantal speed < 20, virtical speed < 40
                self.score += B0 if abs(self.s0[0]) < 20 else -abs(self.s0[0])**2
                self.score += B0 if abs(self.s0[1]) < 40 else -abs(self.s0[1])**2
                assert self.score > 0 , "Assertion error! too much speed penalty !"
                # bonus4 remaining fuel the more the better
                self.score += self.fuel # 不到時候去太強調省油
        
    def move(self, rotate, thrust):
        a1 = angle15(self.a0,rotate)  # 這一步的角度 
        pwr1 = thrust_1(self.pwr0,thrust)  # 這一步的 引擎推力
        force = g + angle2uv(a1) * pwr1   # force on the ship during this step 
        s1 = self.s0 + force  # 這一步將達到的速度
        p1 = self.p0 + self.s0 + (1/2)*(force)  # where "s0 + (1/2)*(force)" is position delta 這一步將到達的位置
        self.fuel -= pwr1
        self.a0 = a1
        self.pwr0 = pwr1
        self.s0 = s1
        self.p0 = p1
        self.chromosome.append([a1,pwr1])  # 這一步的 command 就是這一步的 gene 
        self.trace.append(p1)  # 這一步走完到達的新位置
        assert pwr1 <=4 and pwr1 >=0 , "Assertion error! 引擎推力超過範圍了！(0~4 之間)"
        assert a1 <=90 and a1 >=-90 , "Assertion error! 小艇角度超過範圍了！(-90~90度之間)"
    
    def obsoleted_random_step(self):
        # return the score after this step
        a1 = angle15(self.a0, self.a0 + random.randint(-15, 15))  # 這一步的角度 0~15 之間亂轉
        pwr1 = thrust_1(self.pwr0, self.pwr0 + random.randint(-1,1))  # 這一步的 引擎推力 -1 0 1 亂選
        self.move(a1, pwr1)
        # 結果 self.score = -1 表示已經飛出範圍。
        if (self.p0[1] < 0) or (self.p0[0] < 0) or (self.p0[0] > 7000) :  # higher than 3000 is allowed
            self.score = -1
        else:
            self.make_score()
    
    def explore(self):
        # 用亂數摸索出到達 flate ground 的一條 chromosome
        while(True):
            a1 = angle15(self.a0, self.a0 + random.randint(-15, 15))  # 這一步的角度 0~15 之間亂轉
            pwr1 = thrust_1(self.pwr0, self.pwr0 + random.randint(-1,1))  # 這一步的 引擎推力 -1 0 1 亂選
            self.move(a1, pwr1)
            if (self.p0[1] < 0) or (self.p0[0] < 0) or (self.p0[0] > 7000) :  # higher than 3000 is allowed
                # 結果 self.score = -1 表示已經飛出範圍。
                self.score = -1
            else:
                self.make_score()
            if self.score < 0 or not self.inTheAir: 
                # 用亂數摸索出到達 flate ground 的一條 chromosome 為止。
                break

    def evaluate(self, chromosome):
        # 給這條 chromosome 打分數
        # 若怕用完了還在空中，可重複最後一個命令很多次，或用亂數先加很長很長再送進來試驗。
        # 傳回值是這條 chromosome 的有效長度
        
        for i in range(len(chromosome)):
            self.move(chromosome[i][0], chromosome[i][1])
            if (self.p0[1] < 0) or (self.p0[0] < 0) or (self.p0[0] > 7000) :  # higher than 3000 is allowed
                # 結果 self.score = -1 表示已經飛出範圍。
                self.score = -1
            else:
                self.make_score()
            if self.score < 0 or not self.inTheAir: 
                break
        return i  # lengh of this chromosome


### 做出一個 class 叫作 Ship 
有油料、位置、速度、角度、引擎出力、軌跡、從開始到落地的一長串 chromosome 也就是一長串的基因 genes (rotate,power) 、還有 score。

先亂撒一組命令 > get the distance if alives then repeat else if crash then record the final distance (to the flate ground) as its score. 嘗試的過程不評分，要撞毀了或落地了才評分。
如果 parent 都撞毀了，誰好? 簡化地形為 surface 一律朝上，則越接近 flate ground 的就是越好。若有凹洞, 也就是 surface 有朝下時，要另外想辦法。如果 parent 都撞毀了，但是其子代 gene 都用完了還 inTheAir 就要自己用亂數探索自己的前途，最終要撞毀了或落地了才評分這個子代。經過 crossover and mutation 或單純的 exploration 都一定會跑出雜枝，不影響評分與演化過程因為會被 ship.move() 修正好。


In [5]:
# verification test 
for i in range(5):
    trial_count = 0
    while True:
        p0 = v(2500, 2700)  # Initial position
        a0 = 0        # absolute angle of the ship 
        pwr0 = 0      # 引擎出力
        s0 = v(0,0)     # Initial speed vector
        ship = Ship(p0, s0, a0, pwr0, fuel0)
        ship.explore()
        if ship.score > 0 : 
            print(f"{i} trial count",trial_count)
            print(ship.chromosome)
            print("score", ship.score)
            print()
            break
        trial_count += 1
            

1>

 distances -->


distances --> [2661.1725315883314, 1560.6249526648771, 1452.8874418179216, 1770.224489402474, 3239.2962805029506, 3569.739541322343] (<class 'list'>)
1>

 self :> p0 -->


self :> p0 --> [2500.     2698.1445] (<class 'numpy.ndarray'>)
1>

...

3.0
1>

 exit


1>1>

 bd* exit


1>0 trial count 55
[[-5, 1], [-11, 2], [-7, 1], [-22, 1], [-32, 2], [-26, 3], [-18, 3], [-22, 4], [-35, 4], [-50, 4], [-38, 4], [-45, 4], [-33, 4], [-43, 4], [-53, 3], [-64, 2], [-62, 2], [-59, 1], [-64, 1], [-71, 0], [-64, 1], [-71, 2], [-66, 3], [-66, 3], [-61, 3], [-67, 3], [-66, 2], [-71, 3], [-80, 4], [-90, 4], [-90, 3], [-90, 2], [-90, 3], [-90, 4], [-88, 4], [-83, 4], [-83, 4], [-73, 4], [-79, 4], [-90, 3], [-90, 2], [-80, 1], [-87, 2], [-74, 2], [-89, 2], [-86, 3], [-90, 3], [-90, 4], [-90, 3]]
score 23303.153974419554

1 trial count 15
[[-10, 0], [-8, 1], [-11, 1], [-19, 1], [-23, 2], [-12, 1], [-26, 2], [-27, 3], [-41, 4], [-48, 4], [-56, 4], [-60, 4], [-47, 4], [-61, 4], [-48, 4], [-36, 3], [-37, 3], [-46, 3], [-32, 4], [-30, 4], [-31, 4], [-41, 4], [-49, 3], [-50, 2], [-63, 3], [-57, 2], [-46, 3], [-43, 2], [-35, 3], [-22, 3], [-13, 4], [-22, 4], [-26, 4], [-12, 3], [-7, 4], [-7, 3], [-11, 4], [-9, 4], [-9, 3], [1, 4], [-13, 4], [-19, 3], [-13, 3], [-8, 2], [-15, 1], [-12, 

## 用 Genetic Algorithm 先找出一條成功的路徑

一見面先取 100 或 1K 或 1W 名烏合之眾的前 15 名，兩兩交配湊 105 個下一代，再取前 15 名兩兩配對又得 105 個子代。如此輾轉，看看結果。<br>
一見面先取 100 或 1K 或 1W 名烏合之眾的前 15 名，兩兩交配湊 105 個下一代，再取前 15 名兩兩配對又得 105 個子代。如此輾轉，看看結果。<br>
一見面先取 100 或 1K 或 1W 名烏合之眾的前 15 名，兩兩交配湊 105 個下一代，再取前 15 名兩兩配對又得 105 個子代。如此輾轉，看看結果。<br>
一見面先取 5 名烏合之眾兩兩交配湊 10 個下一代，再取前 5 名兩兩配對又得 10 個子代。如此輾轉，看看結果。<br>

In [4]:
# Get population
# 一見面先取 5 名烏合之眾兩兩交配湊 10 個下一代，再取前 5 名兩兩配對又得 10 個子代。如此輾轉，看看結果。
parents = []
for i in range(5):
    while True:
        p0 = v(2500, 2700)  # Initial position
        a0 = 0        # absolute angle of the ship 
        pwr0 = 0      # 引擎出力
        s0 = v(0,0)     # Initial speed vector
        ship = Ship(p0, s0, a0, pwr0, fuel0)
        ship.explore()
        if ship.score > 0 : 
            parents.append((ship.score, ship.chromosome))
            break

In [5]:
parents

[(54076.490997524554,
  [[-11, 0],
   [-9, 0],
   [-23, 1],
   [-38, 0],
   [-43, 1],
   [-34, 1],
   [-24, 2],
   [-29, 2],
   [-44, 2],
   [-52, 3],
   [-61, 3],
   [-56, 2],
   [-42, 3],
   [-38, 3],
   [-44, 3],
   [-56, 3],
   [-57, 4],
   [-58, 4],
   [-51, 4],
   [-45, 4],
   [-38, 4],
   [-30, 4],
   [-17, 3],
   [-15, 3],
   [-5, 3],
   [7, 3],
   [11, 3],
   [-2, 3],
   [-16, 4],
   [-2, 4],
   [-17, 4],
   [-13, 4],
   [-13, 4],
   [-16, 4],
   [-22, 3],
   [-21, 2],
   [-10, 3],
   [0, 3],
   [11, 4],
   [10, 4],
   [-5, 4],
   [-20, 4],
   [-22, 4],
   [-13, 3],
   [-15, 3],
   [-23, 3],
   [-16, 3],
   [-2, 4],
   [-14, 4],
   [-7, 4],
   [7, 4],
   [-6, 4],
   [-12, 4],
   [-21, 3],
   [-15, 4],
   [-9, 4],
   [-10, 4],
   [3, 4],
   [-1, 4],
   [-4, 4]]),
 (48092.81715492832,
  [[14, 1],
   [5, 2],
   [-8, 1],
   [-1, 1],
   [4, 2],
   [-8, 3],
   [-4, 3],
   [-3, 3],
   [-2, 3],
   [-12, 4],
   [-20, 4],
   [-22, 4],
   [-20, 4],
   [-18, 4],
   [-6, 4],
   [-6, 3],
  

In [6]:
epoch = 0 

In [7]:
# mating 用 parents 排列組合出下一代
spouses = [c for c in combinations(parents, 2)]

In [8]:
# 查看這些配偶
%f spouses count --> :> [0] count --> -->

spouses count --> 10 (<class 'int'>)
spouses count --> :> [0] count --> 2 (<class 'int'>)
spouses count --> :> [0] count --> --> ((54076.490997524554, [[-11, 0], [-9, 0], [-23, 1], [-38, 0], [-43, 1], [-34, 1], [-24, 2], [-29, 2], [-44, 2], [-52, 3], [-61, 3], [-56, 2], [-42, 3], [-38, 3], [-44, 3], [-56, 3], [-57, 4], [-58, 4], [-51, 4], [-45, 4], [-38, 4], [-30, 4], [-17, 3], [-15, 3], [-5, 3], [7, 3], [11, 3], [-2, 3], [-16, 4], [-2, 4], [-17, 4], [-13, 4], [-13, 4], [-16, 4], [-22, 3], [-21, 2], [-10, 3], [0, 3], [11, 4], [10, 4], [-5, 4], [-20, 4], [-22, 4], [-13, 3], [-15, 3], [-23, 3], [-16, 3], [-2, 4], [-14, 4], [-7, 4], [7, 4], [-6, 4], [-12, 4], [-21, 3], [-15, 4], [-9, 4], [-10, 4], [3, 4], [-1, 4], [-4, 4]]), (48092.81715492832, [[14, 1], [5, 2], [-8, 1], [-1, 1], [4, 2], [-8, 3], [-4, 3], [-3, 3], [-2, 3], [-12, 4], [-20, 4], [-22, 4], [-20, 4], [-18, 4], [-6, 4], [-6, 3], [-7, 4], [-12, 4], [-21, 3], [-23, 4], [-34, 4], [-27, 4], [-19, 4], [-17, 3], [-22, 3], [-17, 3], [

In [ ]:
# crossover 產生下一代，隨即令其突變 (從頭逐一 gene 依序突變)
offspring = []
for i in range(len(spouses)):  # 10 個子代都針對某一組 (epoch) gene 突變，大家來較量
    offspring_crossover = crossover(spouses[i])
    if len(offspring_crossover) >= epoch :
        # 不一定 mutate 得到，看長度夠不夠
        offspring_crossover[epoch] = mutation(offspring_crossover[epoch])  # 只 mutate 一個 gene
    offspring.append([0,offspring_crossover])  # _working_

In [ ]:
# 上下兩代組合成新一輪的 parents 
parents = sorted(offspring + parents, key=lambda x:x[0])[-5:]
[row[0] for row in parents]    

### 手動連續跌代 (舊版，全部 gene 一起 mutate)

### 手動連續跌代 (新版，個別 gene 依序逐個 mutate)

In [ ]:
# mating 用 parents 排列組合出成對的配偶
spouses = [c for c in combinations(parents, 2)]

# crossover 產生下一代，隨即令其突變
offspring = []
for i in range(len(spouses)):
    offspring.append([0,list(map(mutation,crossover(spouses[i])))])

# evaluate the offspring populataion
p0 = v(2500, 2700)  # Initial position
a0 = 0        # absolute angle of the ship 
pwr0 = 0      # 引擎出力
s0 = v(0,0)     # Initial speed vector
for i in range(len(offspring)):
    offspring[i][1] = offspring[i][1] + [offspring[i][1][-1]]*200
    ship = Ship(p0, s0, a0, pwr0, fuel0)
    ship.evaluate(offspring[i][1])
    offspring[i][0] = ship.score
    %f i ship :> score -2 slice -->
epoch += 1 
# 上下兩代組合成新一輪的 parents 
parents = sorted(offspring + parents, key=lambda x:x[0])[-5:]
[row[0] for row in parents]    

### 自動連續跌代

In [ ]:
import numpy as np
import re, math, peforth, random, pdb
from itertools import combinations  # combinations 數學「排列組合」中的「組合」
%f version drop

In [ ]:
%run mars.py

# Get population 
# 一見面先取 5 名烏合之眾兩兩交配湊 10 個下一代，再取前 5 名兩兩配對又得 10 個子代。如此輾轉，看看結果。
parents = []
for i in range(5):
    while True:
        p0 = v(2500, 2700)  # Initial position
        a0 = 0        # absolute angle of the ship 
        pwr0 = 0      # 引擎出力
        s0 = v(0,0)     # Initial speed vector
        ship = Ship(p0, s0, a0, pwr0, fuel0)
        ship.explore()
        if ship.score > 0 : 
            parents.append((ship.score, ship.chromosome))
            break
# 開始自動演化
high0 = 0
for j in range(1000):
    # mating 用 parents 排列組合出成對的配偶
    spouses = [c for c in combinations(parents, 2)]

    # crossover 產生下一代，隨即令其突變
    offspring = []
    for i in range(len(spouses)):
        offspring.append([0,list(map(mutation,crossover(spouses[i])))])

    # evaluate the offspring populataion
    p0 = v(2500, 2700)  # Initial position
    a0 = 0        # absolute angle of the ship 
    pwr0 = 0      # 引擎出力
    s0 = v(0,0)     # Initial speed vector
    for i in range(len(offspring)):
        offspring[i][1] = offspring[i][1] + [offspring[i][1][-1]]*200
        ship = Ship(p0, s0, a0, pwr0, fuel0)
        ship.evaluate(offspring[i][1])
        offspring[i][0] = ship.score

    # 上下兩代組合成新一輪的 parents 
    parents = sorted(offspring + parents, key=lambda x:x[0])[-5:]
    high1 = [row[0] for row in parents][4]
    if j%10 == 0 : print(f"#{j}", end=" ") 
    if high0 != high1 : print(f"{j}:{high1}")
    high0 = high1

# evaluate a specific chromosome
p0 = v(2500, 2700)  # Initial position
a0 = 0        # absolute angle of the ship 
pwr0 = 0      # 引擎出力
s0 = v(0,0)     # Initial speed vector
ship = Ship(p0, s0, a0, pwr0, fuel0)
ship.evaluate(parents[4][1])
%f ship :> make_score() ship :> score -->
%f ship :> chromosome -->


# 修正 mutation 的方式
_working_
2019/06/13 17:41 Mutation 的方式不好，咱目前是一次所有的 gene 全部 mutate，可能是造成訓練效果不佳的原因！因為某個 gene 的 mutation 再怎麼好效果被沖淡了，無法彰顯。所以要一組一組從頭照順序來讓 Mutation 找出最佳組合。這樣看來，窮舉是 93 種組合 for 一組 gene (-15~15, -1~1)，我目前是 10 個 offspring 只有 1/9, 將來考慮改多點。窮舉可能不見得好，因為它假設後面的一長串都是對的才有意義，當然這大有疑問，所以要大家一起慢慢演化才好。從頭 mutation 到尾 (每條 chromosome 的長度不一樣，看哪一條最長都要涵蓋到) 就是一輪 epoch. 

### 手動連續跌代 (新版，個別 gene 依序逐個 mutate)

In [ ]:
import numpy as np
import re, math, peforth, random, pdb
from itertools import combinations  # combinations 數學「排列組合」中的「組合」
%f version drop

In [ ]:
%run mars.py

In [ ]:
epoch = 0 
p0 = v(2500, 2700)  # Initial position
a0 = 0        # absolute angle of the ship 
pwr0 = 0      # 引擎出力
s0 = v(0,0)     # Initial speed vector

In [ ]:
# Get population
# 一見面先取 5 名烏合之眾兩兩交配湊 10 個下一代，再取前 5 名兩兩配對又得 10 個子代。如此輾轉，看看結果。
parents = []
for i in range(5):
    while True:
        ship = Ship(p0, s0, a0, pwr0, fuel0)
        ship.explore()
        if ship.score > 0 : 
            parents.append((ship.score, ship.chromosome))
            break

In [ ]:
# mating 用 parents 交配出親代的排列組合
spouses = [c for c in combinations(parents, 2)]

# crossover 產生下一代，隨即令其突變 (從頭逐一 gene 依序突變)
offspring = []
for i in range(len(spouses)):  # 10 個子代都針對某一組 (epoch) gene 突變，大家來較量
    offspring_crossover = crossover(spouses[i])
    if len(offspring_crossover) >= epoch :
        # 不一定 mutate 得到，看長度夠不夠
        offspring_crossover[epoch] = mutation(offspring_crossover[epoch])  # 只 mutate 一個 gene
    offspring.append([0,offspring_crossover])

offspring_length = len(offspring)
epoch_end = 0 # 如果 epoch count 大過所有的 chromosome_length 則 epoch 就該結束了
for i in range(offspring_length):
    offspring[i][1] = offspring[i][1] + [offspring[i][1][-1]]*200  # 重複最後一組 gene 保證使 ship 落地
    ship = Ship(p0, s0, a0, pwr0, fuel0)
    chromosome_length = ship.evaluate(offspring[i][1])
    if epoch > chromosome_length : epoch_end += 1
    offspring[i][0] = ship.score
    %f i ship :> score -2 slice -->
        
if epoch_end >= offspring_length :
    %f ." End of epoch"
    epoch = 0
else:
    epoch += 1
    %f epoch --> 



# 上下兩代組合成新一輪的 parents 
parents = sorted(offspring + parents, key=lambda x:x[0])[-5:]
[row[0] for row in parents]    

### 自動連續跌代 (新版，個別 gene 依序逐個 mutate)

In [10]:
import numpy as np
import re, math, peforth, random, pdb
from itertools import combinations  # combinations 數學「排列組合」中的「組合」
%f version drop
# %run mars.py
%run episode2.py

# Get population
# 一見面先取 5 名烏合之眾兩兩交配湊 10 個下一代，再取前 5 名兩兩配對又得 10 個子代。如此輾轉，看看結果。
parents = []
for i in range(5):
    while True:
        ship = Ship(p0, s0, a0, pwr0, fuel0)
        ship.explore()
        if ship.score > 0 : 
            parents.append((ship.score, ship.chromosome))
            break
high0 = 0            
for cycle in range(1000):
    epoch = 0  # a new cycle 
    while(True):
        # mating 用 parents 交配出親代的排列組合
        spouses = [c for c in combinations(parents, 2)]

        # crossover 產生下一代，隨即令其突變 (從頭逐一 gene 依序突變)
        offspring = []
        for i in range(len(spouses)):  # 10 個子代都針對某一組 (epoch) gene 突變，大家來較量
            offspring_crossover = crossover(spouses[i])
            if len(offspring_crossover) >= epoch :
                # 不一定 mutate 得到，看長度夠不夠
                offspring_crossover[epoch] = mutation(offspring_crossover[epoch])  # 只 mutate 一個 gene
            offspring.append([0,offspring_crossover])

        # 為這些子代評分
        offspring_length = len(offspring)
        epoch_end = 0 # 如果 epoch count 大過所有的 chromosome_length 則 epoch 就該結束了
        for i in range(offspring_length):
            offspring[i][1] = offspring[i][1] + [offspring[i][1][-1]]*200  # 重複最後一組 gene 保證使 ship 落地
            ship = Ship(p0, s0, a0, pwr0, fuel0)
            chromosome_length = ship.evaluate(offspring[i][1])
            if epoch > chromosome_length : epoch_end += 1
            offspring[i][0] = ship.score
                
        # 上下兩代組合成新一輪的 parents 
        parents = sorted(offspring + parents, key=lambda x:x[0])[-5:]
        high1 = parents[4][0]
        if high0 != high1 :
            print(f"cycle:{cycle} epoch:{epoch} highest score:{high1}") 
            high0 = high1

        if epoch_end >= offspring_length :
            print(f"cycle:{cycle} ~~~~ End of epoch ~~~~")
            break
        else:
            epoch += 1

p e f o r t h    v1.24
source code http://github.com/hcchengithub/peforth
Type 'peforth.ok()' to enter forth interpreter, 'exit' to come back.

cycle:0 epoch:0 highest score:51395.68879965464
cycle:0 epoch:1 highest score:51468.26907458854
cycle:0 epoch:4 highest score:51566.747607322664
cycle:0 epoch:5 highest score:51628.600516725
cycle:0 epoch:6 highest score:51810.218217285284
cycle:0 epoch:10 highest score:51841.8404994814
cycle:0 epoch:15 highest score:51856.89165178909
cycle:0 epoch:16 highest score:51902.54248977718
cycle:0 epoch:28 highest score:51903.65732226424
cycle:0 epoch:29 highest score:51910.89756362877
cycle:0 epoch:30 highest score:51920.46308408334
cycle:0 epoch:48 highest score:51925.44303569189
cycle:0 epoch:50 highest score:51953.890241171335
cycle:0 epoch:58 highest score:51962.46761453559
cycle:0 epoch:61 highest score:51971.54283705839
cycle:0 epoch:63 highest score:51995.643615769164
cycle:0 epoch:65 highest score:52006.38686305069
cycle:0 epoch:66 highest sc

KeyboardInterrupt: 

In [1]:
# evaluate a specific chromosome
ship = Ship(p0, s0, a0, pwr0, fuel0)
ship.evaluate(parents[4][1])
%f ship :> make_score() ship :> score -->
%f ship :> chromosome -->
[ np.round(i,0) for i in ship.trace]

NameError: name 'Ship' is not defined

In [2]:
%f words

code end-code \ # // <selftest> </selftest> bye /// unknown immediate stop compyle trim indent -indent <py> </py> </pyV> words . cr help interpret-only compile-only literal reveal privacy (create) : ; ( BL CR word ' , [compile] py: py> py:~ py>~ 0branch here! here swap ! @ ? >r r> r@ drop nip dup over 0< + * - / % 1+ 2+ 1- 2- compile if then compiling char last version execute cls private nonprivate (space) exit ret rescan-word-hash (') branch bool and or not (forget) AND OR NOT XOR true false "" [] {} none >> << 0= 0> 0<> 0<= 0>= = == > < != >= <= abs max min doVar doNext depth pick roll space [ ] colon-word create (marker) marker next abort alias <> public rot -rot 2drop 2dup slice invert negate within ['] allot for begin until again ahead never repeat aft else while ?stop ?dup variable +! chars spaces .( ." .' s" s' s` does> count accept <accept> nop </accept>  case of endof endcase refill [else] [if] [then] (::) (:>) :: :> ::~ :>~ "msg"abort abort" "msg"?abort ?abort" '<text> (<te